In [1]:
!pip install hyperas
!pip install hyperopt

    100% |████████████████████████████████| 348kB 23.6MB/s 
  Running setup.py bdist_wheel for hyperas ... - done
  Stored in directory: /root/.cache/pip/wheels/06/38/3f/27826f57fae60ef788ceb47e2c649590ab8af31f42075325d2
Successfully built hyperas
ipython 5.5.0 has requirement prompt-toolkit<2.0.0,>=1.0.4, but you'll have prompt-toolkit 2.0.7 which is incompatible.
cufflinks 0.14.6 has requirement plotly>=3.0.0, but you'll have plotly 1.12.12 which is incompatible.
  Found existing installation: prompt-toolkit 1.0.15
    Uninstalling prompt-toolkit-1.0.15:
      Successfully uninstalled prompt-toolkit-1.0.15


In [2]:
from __future__ import print_function
from hyperopt import Trials, STATUS_OK, tpe
from hyperas import optim
from hyperas.distributions import choice, uniform
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation
from keras.datasets import mnist
from keras.utils import np_utils

Using TensorFlow backend.


In [0]:
def data():
    '''
    Data providing function:
    This function is separated from model() so that hyperopt
    won't reload data for each evaluation run.
    '''
    (X_train, y_train), (X_test, y_test) = mnist.load_data()
    X_train = X_train.reshape(60000, 784)
    X_test = X_test.reshape(10000, 784)
    X_train = X_train.astype('float32')
    X_test = X_test.astype('float32')
    X_train /= 255
    X_test /= 255
    nb_classes = 10
    Y_train = np_utils.to_categorical(y_train, nb_classes)
    Y_test = np_utils.to_categorical(y_test, nb_classes)
    return X_train, Y_train, X_test, Y_test

In [0]:
def model(X_train, Y_train, X_test, Y_test):
    '''
    Model providing function:
    Create Keras model with double curly brackets dropped-in as needed.
    Return value has to be a valid python dictionary with two customary keys:
        - loss: Specify a numeric evaluation metric to be minimized
        - status: Just use STATUS_OK and see hyperopt documentation if not feasible
    The last one is optional, though recommended, namely:
        - model: specify the model just created so that we can later use it again.
    '''
    model = Sequential()
    model.add(Dense(512, input_shape=(784,)))
    model.add(Activation('relu'))
    model.add(Dropout({{uniform(0, 1)}}))
    model.add(Dense({{choice([256, 512, 1024])}}))
    model.add(Activation({{choice(['relu', 'sigmoid'])}}))
    model.add(Dropout({{uniform(0, 1)}}))

    # If we choose 'four', add an additional fourth layer
    if {{choice(['three', 'four'])}} == 'four':
        model.add(Dense(100))
        model.add({{choice([Dropout(0.5), Activation('linear')])}})
        model.add(Activation('relu'))

    model.add(Dense(10))
    model.add(Activation('softmax'))

    model.compile(loss='categorical_crossentropy',
                  optimizer={{choice(['rmsprop', 'adam', 'sgd'])}},
                  metrics=['accuracy'])
        
    model.fit(X_train, Y_train,
              batch_size={{choice([64, 128])}},
              epochs=1,
              verbose=2,
              validation_data=(X_test, Y_test))
    score, acc = model.evaluate(X_test, Y_test, verbose=0)
    print('Test accuracy:', acc)
    return {'loss': -acc, 'status': STATUS_OK, 'model': model}

In [6]:
trials = Trials()
best_run, best_model = optim.minimize(model=model, # the model
                                          data=data, # the data generating function
                                          max_evals=10, # number of iterations
                                          algo=tpe.suggest,
                                          notebook_name='Hyperas_testing', # This is important! Name of the notebook
                                          trials=trials, # contains parameters and results for all runs
                                          eval_space=True
                                     )

>>> Imports:
#coding=utf-8

from __future__ import print_function

try:
    from hyperopt import Trials, STATUS_OK, tpe
except:
    pass

try:
    from hyperas import optim
except:
    pass

try:
    from hyperas.distributions import choice, uniform
except:
    pass

try:
    from keras.models import Sequential
except:
    pass

try:
    from keras.layers.core import Dense, Dropout, Activation
except:
    pass

try:
    from keras.datasets import mnist
except:
    pass

try:
    from keras.utils import np_utils
except:
    pass

try:
    import pandas as pd
except:
    pass

>>> Hyperas search space:

def get_space():
    return {
        'Dropout': hp.uniform('Dropout', 0, 1),
        'Dense': hp.choice('Dense', [256, 512, 1024]),
        'Activation': hp.choice('Activation', ['relu', 'sigmoid']),
        'Dropout_1': hp.uniform('Dropout_1', 0, 1),
        'Dropout_2': hp.choice('Dropout_2', ['three', 'four']),
        'add': hp.choice('add', [Dropout(0.5), Activation('linear')]),
   

In [9]:
# Get all the results ordered
import pandas as pd
df = pd.DataFrame()
results =[]
for trial in trials.trials:
        vals = trial.get('misc').get('vals')
        vals['result'] = - trial.get('result').get('loss')
        results.append(vals)

df = pd.DataFrame(results)
df.sort_values(by='result',ascending = False).head(1)

,Activation,Dense,Dropout,Dropout_1,Dropout_2,add,batch_size,optimizer,result
9,[1],[2],[0.4143619965361732],[0.09225974322037533],[1],[1],[0],[1],0.9632


In [8]:
best_run

{'Activation': 'sigmoid',
 'Dense': 1024,
 'Dropout': 0.4143619965361732,
 'Dropout_1': 0.09225974322037533,
 'Dropout_2': 'four',
 'add': <keras.layers.core.Activation at 0x7fd2438b55f8>,
 'batch_size': 64,
 'optimizer': 'adam'}